In [1]:
using Pkg
Pkg.instantiate()
using JuMP
using GLPK
using DataFrames

## Creating tables for the Data

In [2]:
cost_and_effectiveness = DataFrame(Vaccine_Type=[:A, :B, :C, :D], 
                           Cost=[2., 5., 3., 4.],
                           Effectiveness=[.50,.70,.60,.80])

,Vaccine_Type,Cost,Effectiveness
,Symbol,Float64,Float64
1,A,2.0,0.5
2,B,5.0,0.7
3,C,3.0,0.6
4,D,4.0,0.8


In [4]:
cost_per_vaccine_per_time = DataFrame(Vaccine_Type=[:A, :B, :C, :D],
                                                T₀=[2., 5., 3., 4],
                                                T₁=[1.8, 4.5, 2.7, 3.6],
                                                T₂=[1.62, 4.05, 2.43, 3.24],
                                                T₃=[1.458, 3.645, 2.187, 2.916],
                                                T₄=[1.3122, 3.2805, 1.9683, 2.6244],
                                                T₅=[1.18098, 2.95245, 1.77147, 2.36196 ],
                                                T₆=[1.062882, 2.657205, 1.594323, 2.125764],
                                                T₇=[0.956594, 2.391485, 1.4348, 1.913188])

,Vaccine_Type,T₀,T₁,T₂,T₃,T₄,T₅,T₆,T₇
,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,A,2.0,1.8,1.62,1.458,1.3122,1.18098,1.06288,0.956594
2,B,5.0,4.5,4.05,3.645,3.2805,2.95245,2.6572,2.39148
3,C,3.0,2.7,2.43,2.187,1.9683,1.77147,1.59432,1.4348
4,D,4.0,3.6,3.24,2.916,2.6244,2.36196,2.12576,1.91319


In [5]:
population = DataFrame(Age_range=["0-5", "6-18", "19-65","65+"],
                       Population=[19200, 54400, 268400, 58000])

,Age_range,Population
,String,Int64
1,0-5,19200
2,6-18,54400
3,19-65,268400
4,65+,58000


In [ ]:
effectiveness_per_population = DataFrame(Age_range=["0-5", "6-18", "19-65","65+"],
                                        A=[.75, .50, .50, .50],
                                        B=[.70,.85, .70, .70],
                                        C=[.60,.60, .5, .60],
                                        D=[.65,.65, .65, .65])

,Age_range,A,B,C,D
,String,Float64,Float64,Float64,Float64
1,0-5,0.75,0.7,0.6,0.65
2,6-18,0.5,0.85,0.6,0.65
3,19-65,0.5,0.7,0.5,0.65
4,65+,0.5,0.7,0.6,0.8


In [7]:
budget_per_time = DataFrame(Time=[:T₀, :T₁, :T₂, :T₃, :T₄, :T₅, :T₆, :T₇ ],
                            Budget=[142850., 142850., 142850., 142850., 142850., 142850., 142850., 142850.])

,Time,Budget
,Symbol,Float64
1,T₀,142850.0
2,T₁,142850.0
3,T₂,142850.0
4,T₃,142850.0
5,T₄,142850.0
6,T₅,142850.0
7,T₆,142850.0
8,T₇,142850.0


In [13]:
function resultant_value(vaccine_type)
    col_ind = findall(x->x==vaccine_type, cost_and_effectiveness[:,names(cost_and_effectiveness)[1]])[1]
    êᵢ = cost_and_effectiveness[:,:Effectiveness][col_ind]
    Σ = sum([e*n for (e,n) in zip(effectiveness_per_population[:, vaccine_type], population[:,:Population])])
    eᵢ = êᵢ * Σ
    return eᵢ
end

resultant_value (generic function with 1 method)

In [10]:
function find_col_index(var, df) 
    ind = findall(x->x==var, df[:,names(df)[1]])[1]
    return ind
end

find_col_index (generic function with 1 method)

## Solving the LP

In [38]:
# V = [resultant_value(vax) for vax in [:A, :B, :C, :D]]
# C = cost_per_vaccine_per_time[:,:T₀]
# B = budget_per_time[:,:Budget][find_col_index(:T₀, budget_per_time)]

# N = length(cost_and_effectiveness[:,:Vaccine_Type])

4

In [41]:
# tvlp = Model(GLPK.Optimizer)
# init = [0 for i=1:N]

# @variable(tvlp, x[i=1:N] >= 0, Int, start = init[i])
# @constraint(tvlp, C'x <= B)
# @objective(tvlp, Max, V'x)
# optimize!(tvlp)
# value.(x)

## TV-LP solver

In [14]:
xs = []
ts = [:T₀, :T₁, :T₂, :T₃, :T₄, :T₅, :T₆, :T₇]
V = [resultant_value(vax) for vax in [:A, :B, :C, :D]]
N = length(cost_and_effectiveness[:,:Vaccine_Type])
init = [0 for i=1:N]


for t in ts 
    C = cost_per_vaccine_per_time[:,t]
    B = budget_per_time[:,:Budget][find_col_index(t, budget_per_time)] 

    tvlp = Model(GLPK.Optimizer) 

    @variable(tvlp, x[i=1:N] >= 0, Int, start = init[i])
    @constraint(tvlp, C'x <= B)
    @objective(tvlp, Max, V'x)
    optimize!(tvlp)
    init = value.(x)
    push!(xs, init)
    println("Time step ",t," ",init)
end


Time step T₀
Time step T₁
Time step T₂
Time step T₃


In [12]:
xs

8-element Vector{Any}:
 [1.0, 0.0, 0.0, 35712.0]
 [1.0, 0.0, 0.0, 39680.0]
 [1.0, 0.0, 0.0, 44089.0]
 [1.0, 0.0, 1.0, 48987.0]
 [1.0, 0.0, 1.0, 54430.0]
 [1.0, 0.0, 1.0, 60478.0]
 [1.0, 0.0, 1.0, 67198.0]
 [0.0, 0.0, 1.0, 74665.0]

In [33]:
V

4-element Vector{Float64}:
 102.39999999999999
 201.71200000000002
 168.15599999999998
 214.96000000000004